In [3]:
from os import path
import pymongo
import pickle
import glob
import math
import numpy
import scipy.stats
import lxml.etree as lxml
import matplotlib
import operator

years = [[2015], [2016], [2017], [2018], [2019],[2020]
         , [2015, 2016], [2016, 2017], [2017, 2018], [2018, 2019], [2019,2020]
         ,[2015, 2016, 2017], [2016, 2017, 2018], [2017, 2018, 2019], [2018, 2019, 2020]
         , [2015, 2016, 2017, 2018], [2016, 2017, 2018, 2019], [2017, 2018, 2019, 2020]
         , [2015, 2016, 2017, 2018, 2019], [2016, 2017, 2018, 2019, 2020]]

state_gov_nm = 'CA'

data_file_location = 'C:/Users/William Mitchell/Documents/QuigleyData'

_pol_party = path.join(data_file_location,state_gov_nm + 'pol_party.dict')

_sponsor_table = path.join(data_file_location,state_gov_nm + 'sponsor_table.dict')

_temp_sponsor_table = path.join(data_file_location,state_gov_nm + 'temp_sponsor_table.dict')

_d_sponsor_table = path.join(data_file_location,state_gov_nm + 'd_sponsor_table.dict')
    
_d_temp_sponsor_table = path.join(data_file_location,state_gov_nm + 'd_temp_sponsor_table.dict')

_district_leadership = path.join(data_file_location,state_gov_nm + 'district_leadership.dict')

_people_leadership = path.join(data_file_location,state_gov_nm + 'people_leadership.dict')

_people_ideology = path.join(data_file_location,state_gov_nm + 'people_ideology.dict')

In [4]:
#altered the code from https://github.com/govtrack/govtrack.us-web/blob/master/analysis/sponsorship_analysis.py
def smooth_matrix(nreps, P):

    for i in range(nreps):
        for j in range(nreps):
            P[i,j] = math.sqrt(P[i,j])

def onenorm(u):
    return sum(abs(u))

def rescale(u, log=False):

    u = (u - min(u)) / (max(u) - min(u))


    if log:
        m = numpy.median(u)
        s = -m**2/(2*m - 1)
        if s > 0:
            u = numpy.log(u + s)
            u = (u - min(u)) / (max(u) - min(u))
    return [float(v) for v in u]

def leadership_analysis(state, year, chamber, state_and_year_matrices):
    
    P = state_and_year_matrices[state][year][chamber]
    
    nreps = state_and_year_matrices[state][year][chamber].shape[0]
    for col in range(nreps):
        s = sum(P[:,col])
        if s == 0:
            #raise ValueError()
            #print("Error")
            pass
        if s < 10:
            P[:,col] += (10.0-s)/nreps
            s = 10
        P[:,col] = P[:,col] / s
    v = numpy.ones( (nreps, 1) ) / float(nreps)
    c = 0.85
    x = numpy.ones( (nreps, 1) ) / float(nreps)
    
    while True:
        y = c * numpy.dot(P, x)
        w = onenorm(x) - onenorm(y)
        y = y + w*v
        err = onenorm(y-x)
        if err < .00000000001:
            break
        x = y

    x = rescale(x, log=True)

    return x # this is the pagerank

def ideology_analysis(state, year, chamber, state_and_year_matrices, pol_party):
    P = state_and_year_matrices[state][year][chamber]
    nreps = state_and_year_matrices[state][year][chamber].shape[0]

    if chamber == "H":
        
        u, s, vh = numpy.linalg.svd(P)
        spectrum = vh[1,:]
        
        R_scores = [spectrum[i] for i in range(nreps) if pol_party[state_and_year_matrices[state][year]['H_ind'][i]] == 2]
        R_score_mean = sum(R_scores)/len(R_scores)
        spectrum = spectrum * R_score_mean

        spectrum = rescale(spectrum)
        
        return spectrum
    
    if chamber == "S":
        
        u, s, vh = numpy.linalg.svd(P)
        spectrum = vh[1,:]
        
        R_scores = [spectrum[i] for i in range(nreps) if pol_party[state_and_year_matrices[state][year]['S_ind'][i]] == 2]
        R_score_mean = sum(R_scores)/len(R_scores)
        spectrum = spectrum * R_score_mean


        spectrum = rescale(spectrum)
        
        return spectrum
        
# functions for time intervals of leadership/ideology
def leadership_analysis_v2(nreps, P, list_of_id):
    for col in range(nreps):
        s = sum(P[:,col])
        if s == 0:
            
            #print("Error")
            pass
        if s < 10:
            P[:,col] += (10.0-s)/nreps
            s = 10
        P[:,col] = P[:,col] / s
    v = numpy.ones( (nreps, 1) ) / float(nreps)
    c = 0.85
    x = numpy.ones( (nreps, 1) ) / float(nreps)
    
    while True:
        y = c * numpy.dot(P, x)
        w = onenorm(x) - onenorm(y)
        y = y + w*v
        err = onenorm(y-x)
        if err < .00000000001:
            break
        x = y

    x = rescale(x, log=True)

    if len(x) != len(list_of_id):
        print("ERROR in leadership")
    
    res = {}
    
    for i in range(len(x)):
        res[list_of_id[i]] = x[i]

    ret = dict( sorted(res.items(), key=operator.itemgetter(1),reverse=True))
    return ret

def ideology_analysis_v2(nreps, P, list_of_id, party_list):
    u, s, vh = numpy.linalg.svd(P)
    spectrum = vh[1,:]

    R_scores = [spectrum[i] for i in range(nreps) if party_list[i] == 2]
    R_score_mean = sum(R_scores)/len(R_scores)
    spectrum = spectrum * R_score_mean

    spectrum = rescale(spectrum)
    
    res = {}
    
    for i in range(len(spectrum)):
        res[list_of_id[i]] = spectrum[i]
    if len(spectrum) != len(list_of_id):
        print("ERROR in ideology")
    ret = {}
    ret = dict(sorted(res.items(), key=operator.itemgetter(1),reverse=True))
    return ret



# matrix construction for time intervals and district
def d_build_matrix(list_of_years, d_temp_sponsor_table, d_sponsor_table):
    res = {}
    total_list_of_sponsors = []
    for t in list_of_years:
        for district in d_temp_sponsor_table[t]['list_of_sponsors']:
            if not district in total_list_of_sponsors:
                total_list_of_sponsors.append(district)


    total_list_of_sponsors.sort()
    
 

    nxn = len(total_list_of_sponsors)

    _P = numpy.identity(nxn, numpy.float)

    res = {}
    ordered_id = []
    
    
    for i, val in enumerate(total_list_of_sponsors): 
        res[val] = {"index": i}
        ordered_id.append(val)
    

    for t in list_of_years:
        for bill in d_sponsor_table[t]:
            if d_sponsor_table[t][bill]['co'] != []:
                for row in d_sponsor_table[t][bill]['primary']:
                    for col in d_sponsor_table[t][bill]['co']:
                        _P[res[row]['index'], res[col]['index']] += 1.0

    return nxn, _P, ordered_id

# matrix construction for time intervals
def build_matrix(list_of_years, temp_sponsor_table, pol_party, sponsor_table):
    res = {}
    total_list_of_sponsors = []
    for t in list_of_years:
        for person in temp_sponsor_table[t]['list_of_sponsors']:
            if not person in total_list_of_sponsors:
                total_list_of_sponsors.append(int(person))


    total_list_of_sponsors.sort()
    

    nxn = len(total_list_of_sponsors)

    _P = numpy.identity(nxn, numpy.float)

    res = {}
    ordered_id = []
    ordered_party = []
    
    for i, val in enumerate(total_list_of_sponsors): 
        res[val] = {"index": i, "party": pol_party[val]}
        ordered_id.append(val)
        ordered_party.append(pol_party[val])


    for t in list_of_years:
        for bill in sponsor_table[t]:  
            if sponsor_table[t][bill]['co'] != []:
                for row in sponsor_table[t][bill]['primary']:
                    for col in sponsor_table[t][bill]['co']:
                        _P[res[row]['index'], res[col]['index']] += 1.0

    return nxn, _P, ordered_id, ordered_party

    

In [6]:
with open(_pol_party, 'rb') as f:
    pol_party = pickle.load(f)
    
with open(_sponsor_table, 'rb') as f:
    sponsor_table = pickle.load(f)

with open(_temp_sponsor_table, 'rb') as f:
    temp_sponsor_table = pickle.load(f)
    
with open(_d_sponsor_table, 'rb') as f:
    d_sponsor_table = pickle.load(f)
    
with open(_d_temp_sponsor_table, 'rb') as f:
    d_temp_sponsor_table = pickle.load(f)

In [7]:
#d_temp_sponsor_table
#d_sponsor_table
#temp_sponsor_table
#pol_party
#sponsor_table

for time in years:
    key = ""
    for i in range(len(time)):
        if i == len(time) - 1:
            key += str(time[i])
        else:
            key += str(time[i]) + "-"
    
    #build district leadership
    d_numrows, D_mat, d_order = d_build_matrix(time, d_temp_sponsor_table, d_sponsor_table)

    #build people leadership
    numrows, P_mat, order, party = build_matrix(time, temp_sponsor_table, pol_party, sponsor_table)
    
    
    
    district_leadership = {}
    people_leadership = {}
    people_ideology = {}

    
    district_leadership[key] = leadership_analysis_v2(d_numrows, D_mat, d_order)
    
    #build people leadership
    people_leadership[key] = leadership_analysis_v2(numrows, P_mat, order)
    
    #build people ideology
    people_ideology[key] = ideology_analysis_v2(numrows, P_mat, order, party)
    


ValueError: min() arg is an empty sequence

In [8]:
with open(_district_leadership, 'wb') as f:
    pickle.dump(district_leadership, f)
with open(_people_leadership, 'wb') as f:
    pickle.dump(people_leadership, f)
with open(_people_ideology, 'wb') as f:
    pickle.dump(people_ideology, f)
